This file uses the KEGG API to make a request for Brite ID data and then stores briteIDs and their corresponding functional annotation groups, subgroups, and annotations to a dataframe that will be accessed later to create a hierarchical dataframe with gene expression values (in hierarchy.ipynb).  

In [68]:
import requests
import pandas as pd

url = 'http://rest.kegg.jp/get/br:br08902'
request = requests.get(url, allow_redirects=True)
open('KEGG_brite_groups.txt', 'wb').write(request.content)
del request

In [69]:
df = pd.DataFrame(columns=['Functional Annotation Group','Functional Annotation Subgroup','keggBriteId','Functional Annotation'])

f = open('KEGG_brite_groups.txt', 'r')
lines = f.readlines()
row_ind = 0
col_ind = 0
while(row_ind<len(lines)):
    #get functional annotation group
    if(lines[row_ind][0:1]=='A'): 
        A = lines[row_ind][1:-1]
        row_ind+=1 # next line
        
        #get functional annotation subgroup
        while(row_ind<len(lines) and lines[row_ind][0:1]!='A'):
            if(lines[row_ind].split('  ')[0]=='B'):
                B = lines[row_ind].split('  ')
                B = B[1][:-1]
                row_ind += 1 # next line
                
                #get briteID and the annotation itself and update dataframe with these values
                while(row_ind<len(lines) and lines[row_ind].split('    ')[0]=='C'):
                    C = lines[row_ind].split('    ')
                    C = C[1][:-1]
                    briteID =  C.split("  ")[0][3:]
                    annotation = C.split("  ")[1]
                    df.loc[col_ind] = [A, B, briteID, annotation]
                    col_ind+=1
                    row_ind += 1 
            else: 
                row_ind+=1
    else: 
        row_ind+=1


In [70]:
df

,Functional Annotation Group,Functional Annotation Subgroup,keggBriteId,Functional Annotation
0,Pathway and Brite,Pathway maps,8901,KEGG pathway maps
1,Pathway and Brite,Brite files,8902,BRITE hierarchy files
2,Pathway and Brite,Brite files,8904,BRITE table files
3,Pathway and Brite,Networks,8907,KEGG networks
4,Genes and Proteins,Orthologs and modules,0001,KEGG Orthology (KO)
...,...,...,...,...
108,Organisms and Cells,Organisms,8603,Plant classification
109,Organisms and Cells,Organisms,8605,Plant pathogens
110,Organisms and Cells,Organisms,8607,Endosymbionts
111,Organisms and Cells,"Cells, tissues, and organs",8501,Human organ systems


In [71]:
df.to_csv('KEGG_brite.csv')